In [3]:
import json
import os
import uuid

import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

In [4]:
!python -m torch.distributed.launch --nproc_per_node=8 --nnodes=1 --node_rank=0 \
        /home/ec2-user/src/pytorch_resnet_cifar10_mirror/trainer.py --use_adascale --num_epochs 200 \
        --batch_size 256 \
        --weight_decay 1e-4

/home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/python: Error while finding module specification for 'torch.distributed.launch' (ModuleNotFoundError: No module named 'torch')


In [ ]:
!  mkdir -p data && cd data && wget -c --quiet https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz && tar -xvzf cifar-10-python.tar.gz

In [ ]:
! mkdir -p saved_models

## Run locally multiGPU training

In [2]:
!python -m torch.distributed.launch --nproc_per_node=8 --nnodes=1 --node_rank=0 \
        /home/ec2-user/SageMaker/pytorch_resnet_cifar10_mirror/cifar.py  --num_epochs 200 \
        --batch_size 256 

/home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/python: Error while finding module specification for 'torch.distributed.launch' (ModuleNotFoundError: No module named 'torch')


In [ ]:
# Run Sagemaker ddp training

In [ ]:
! pwd

In [ ]:
! ls /home/ec2-user/SageMaker/pytorch_resnet_cifar10_mirror/sm


In [1]:
! cd /home/ec2-user/SageMaker/pytorch_resnet_cifar10_mirror/sm &&  python ddp-launcher.py --gpus 8 \
--data_dir /home/ec2-user/SageMaker/data/ \
--model_dir /home/ec2-user/SageMaker/ \
--num_epochs 100




### Data directory:  ['cifar-10-batches-py', 'cifar-10-python.tar.gz']
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
NCCL version 2.7.8+cuda10.1
 ### data_dir  /home/ec2-user/SageMaker/data/
 ### data_dir  /home/ec2-user/SageMaker/data/
 ### data_dir  /home/ec2-user/SageMaker/data/
 ### data_dir  /home/ec2-user/SageMaker/data/
 ### data_dir  /home/ec2-user/SageMaker/data/
 ### data_dir  /home/ec2-user/SageMaker/data/
 ### data_dir  /home/ec2-user/SageMaker/data/
 ### data_dir  /home/ec2-user/SageMaker/data/
Local Rank: 2, Epoch: 0, Training ...
Local Rank: 7, Epoch: 0, Training ...
Local Rank: 3, Epoch: 0, Training ...
Local Rank: 1, Epoch: 0, Training ...
Local Rank: 0, Epoch: 0, Training ...
Local Rank: 4, Epoch: 0, Training ...
Local Rank:

## Multinode multi GPU

In [3]:
config = {
    'batch_size': 256,
    'num_epochs' : 50}

In [4]:
bucket = 'mansmane-us-west-2'


In [8]:
# Training time of this job 
token = str(uuid.uuid4())[:10]  # we create a unique token to avoid checkpoint collisions in S3

job = PyTorch(
    entry_point='ddp-launcher.py',
    source_dir='/home/ec2-user/SageMaker/pytorch_resnet_cifar10_mirror/sm',
    role=get_execution_role(),
    framework_version='1.8.1',
    instance_count=1,
    instance_type='ml.p3.16xlarge',
    base_job_name='resnet-multi-GPU-g5',
    py_version='py36',
    hyperparameters=config,
    checkpoint_s3_uri='s3://{}/{}/checkpoints'.format(bucket, token),  # S3 destination of /opt/ml/checkpoints files
    output_path='s3://{}/{}'.format(bucket, token),
    code_location='s3://{}/{}/code'.format(bucket, token), # source_dir code will be staged in S3 there
    environment={"SMDEBUG_LOG_LEVEL":"off"},  # reduce verbosity of Debugger
    debugger_hook_config=False,  # deactivate debugger to avoid warnings in model artifact
    disable_profiler=True,  # keep running resources to a minimum to avoid permission errors
    metric_definitions=[
        {"Name": "Train_loss", "Regex": "Training_loss: ([0-9.]+).*$"},
        {"Name": "Learning_rate", "Regex": "learning rate: ([0-9.]+).*$"},        
        {"Name": "Val_loss", "Regex": "Val_loss: ([0-9.]+).*$"},        
        {"Name": "Throughput", "Regex": "Throughput: ([0-9.]+).*$"},
        {"Name": "Val_pixel_acc", "Regex": "Val_pixel_acc: ([0-9.]+).*$"}
    ],
    tags=[{'Key': 'Project', 'Value': 'A2D2_segmentation'}])  # tag the job for experiment tracking

In [9]:
train_path = 's3://mansmane-us-west-2/cifar10/'

In [10]:
job.fit({'dataset': train_path}, wait=False)

In [12]:
token = str(uuid.uuid4())[:10]  # we create a unique token to avoid checkpoint collisions in S3
instance_count = 2

job = PyTorch(
    entry_point='ddp-launcher.py',
    source_dir='/home/ec2-user/SageMaker/pytorch_resnet_cifar10_mirror/sm',
    role=get_execution_role(),
    framework_version='1.8.1',
    instance_count=instance_count,
    instance_type='ml.p3.16xlarge',
    base_job_name='resnet-multi-GPU-g5-instance-' + str(instance_count),
    py_version='py36',
    hyperparameters=config,
    checkpoint_s3_uri='s3://{}/{}/checkpoints'.format(bucket, token),  # S3 destination of /opt/ml/checkpoints files
    output_path='s3://{}/{}'.format(bucket, token),
    code_location='s3://{}/{}/code'.format(bucket, token), # source_dir code will be staged in S3 there
    environment={"SMDEBUG_LOG_LEVEL":"off"},  # reduce verbosity of Debugger
    debugger_hook_config=False,  # deactivate debugger to avoid warnings in model artifact
    disable_profiler=True,  # keep running resources to a minimum to avoid permission errors
    metric_definitions=[
        {"Name": "Train_loss", "Regex": "Training_loss: ([0-9.]+).*$"},
        {"Name": "Learning_rate", "Regex": "learning rate: ([0-9.]+).*$"},        
        {"Name": "Val_loss", "Regex": "Val_loss: ([0-9.]+).*$"},        
        {"Name": "Throughput", "Regex": "Throughput: ([0-9.]+).*$"},
        {"Name": "Val_pixel_acc", "Regex": "Val_pixel_acc: ([0-9.]+).*$"}
    ],
    tags=[{'Key': 'Project', 'Value': 'A2D2_segmentation'}])  # tag the job for experiment tracking

train_path = 's3://mansmane-us-west-2/cifar10/'
job.fit({'dataset': train_path}, wait=False)

In [ ]:
! pip install fairscale

     |████████████████████████████████| 240 kB 6.9 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████▎   | 777.8 MB 95.2 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 881.9 MB 7.8 kB/s             
  Created wheel for fairscale: filename=fairscale-0.4.5-py3-none-any.whl size=299652 sha256=fab78c7163c9400e55e41affb8d2d419028ed6d79cc046c64972dcef3d15be05
  Stored in directory: /home/ec2-user/.cache/pip/wheels/77/41/f5/d42db16aa9cfafa47a9c19c7fd5d9b9e8d2dc523fa9ff94344
Successfully built fairscale
  Attempting uninstall: torch
    Found existing installation: torch 1.5.1
    Uninstalling torch-1.5.1:
      Successfully uninstalled torch-1.5.1
